In [1]:
# Effect of multiprocessing

import random
import time
from multiprocessing import Process
from os import getpid

def count_time(a_str):
    time_elapse = random.randint(1, 5)
    print('PID is %d.' % getpid())
    time.sleep(time_elapse)
    print('time for %s is %d s.' % (a_str, time_elapse))
    
def main():
    t0 = time.time()
    count_time('aaa')
    count_time('bbb')
    print('total time is %.2f s.' % (time.time() - t0))
          
# Multiprocessing
def main_multi():
    t0 = time.time()
    p1 = Process(target=count_time, args=('aaa', ))
    p2 = Process(target=count_time, args=('bbb', ))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(p1.pid)
    print(p2.pid)
    print('total time is %.2f s.' % (time.time() - t0))

if __name__ == '__main__':
    main()
    main_multi()

PID is 8028.
time for aaa is 2 s.
PID is 8028.
time for bbb is 3 s.
total time is 5.00 s.
13188
16108
total time is 0.12 s.


In [2]:
# Try the example 2
from multiprocessing import Process
from time import sleep

counter = 0


def sub_task(string):
    global counter
    while counter < 10:
        print(string, end='', flush=True)
        counter += 1
        sleep(0.01)


# Have a counter variable in each sub-process
# 10 'Ping' and 10 'Pong'
# Use Queue

def main():
    Process(target=sub_task, args=('Ping', )).start()
    Process(target=sub_task, args=('Pong', )).start()

if __name__ == '__main__':
    main()

In [3]:
# Effect of multithreading

import random
import time
from threading import Thread
from os import getpid

# Define a class inherited from Thread
class count_time(Thread):
    def __init__(self, a_str):
        super().__init__()
        self._name = a_str
    
    def run(self):
        time_elapse = random.randint(1, 5)
        print('PID is %d.' % getpid())
        time.sleep(time_elapse)
        print('time for %s is %d s.' % (self._name, time_elapse))

# Multithreading
def main():
    t0 = time.time()
    t1 = count_time('aaa')
    t2 = count_time('bbb')
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print('total time is %.2f s.' % (time.time() - t0))
    
if __name__ == '__main__':
    main()

PID is 8028.
PID is 8028.
time for aaa is 1 s.
time for bbb is 2 s.
total time is 2.01 s.


In [4]:
# Use of Lock in multithreading
import time
from threading import Thread, Lock

class Account(object):
    def __init__(self):
        self._balance = 0
        self._lock = Lock()
        
    def deposit(self, money):
        # Once a thread has acquired it, subsequent attempts to acquire it block, until it is released
        self._lock.acquire()
        try:
            new_balance = self._balance + money
            time.sleep(0.001)
            self._balance = new_balance
        # Make sure released in the finally block
        finally:
            self._lock.release()
    
    @property
    def balance(self):
        return self._balance
    
class deposit_threads(Thread):
    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money
    
    def run(self):
        self._account.deposit(self._money)
        
def main():
    account = Account()
    threads = []
    for i in range(100):
        t = deposit_threads(account, 1)
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print('total money in the account is %d.' % account.balance)
    
if __name__ == '__main__':
    main()

total money in the account is 100.
